In [96]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt
# %matplotlib inline
import seaborn as sns; sns.set()
import os
import pickle

In [97]:
pd.set_option('display.max_rows', 100)

from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso

from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

import random 
import math

from sklearn.decomposition import PCA

from sklearn import preprocessing

import pandas_profiling

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import GenericUnivariateSelect, SelectFdr

import gc

from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import f_classif


from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import LeaveOneOut

from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import RobustScaler

from random import seed
from random import randint
from imblearn.over_sampling import SMOTE

from sklearn import metrics

from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.metrics import precision_recall_fscore_support
from pandas.testing import assert_frame_equal
from sklearn.preprocessing import OneHotEncoder

In [98]:
INPUT_PATH="data_synthea_tst/"
MODEL_PATH="Model/"
OUTPUT_PATH="Predictions/"

In [99]:
train_condition = pd.read_csv(INPUT_PATH+"ordcovid_preindexconditions.csv").sort_values(by=['patienticn']).set_index(['patienticn'])

In [100]:
x=[1]+list(range(3,train_condition.shape[1]-2))
train_fm=train_condition.iloc[:, x]

In [101]:
train_medications = pd.read_csv(INPUT_PATH+"ordcovid_preindexmedications.csv").sort_values(by=['patienticn']).set_index(['patienticn'])

In [102]:
assert_frame_equal(train_fm[["everpositive"]], train_medications[["everpositive"]])

In [103]:
train_fm = pd.concat([train_fm, train_medications.iloc[:,range(3,train_medications.shape[1]-2)]], axis=1)

In [104]:
train_procedures = pd.read_csv(INPUT_PATH+"ordcovid_preindexprocedures.csv").sort_values(by=['patienticn']).set_index(['patienticn'])

In [105]:
assert_frame_equal(train_fm[["everpositive"]], train_procedures[["everpositive"]])

In [106]:
train_fm = pd.concat([train_fm, train_procedures.iloc[:,range(3,train_procedures.shape[1]-2)]], axis=1)

In [107]:
train_symptoms = pd.read_csv(INPUT_PATH+"ordcovid_preindexsymptoms.csv").sort_values(by=['patienticn']).set_index(['patienticn'])

In [108]:
assert_frame_equal(train_fm[["everpositive"]], train_symptoms[["everpositive"]])

In [109]:
train_fm = pd.concat([train_fm, train_symptoms.iloc[:,range(3,train_symptoms.shape[1]-2)]], axis=1)

In [110]:
train_patient = pd.read_csv(INPUT_PATH+"ordcovid_patient.csv").sort_values(by=['patienticn']).set_index(['patienticn'])

In [111]:
assert_frame_equal(train_fm[["everpositive"]], train_patient[["everpositive"]])

In [112]:
train_fm = pd.concat([train_fm, train_patient.iloc[:,range(19,train_patient.shape[1]-2)]], axis=1)

In [113]:
le = pickle.load(open(MODEL_PATH+"Gender_encoder", 'rb'))
sex_encode=le.transform(train_patient["sex"])

In [114]:
dataframe=pd.DataFrame(sex_encode, columns=["sex"]) 
dataframe['patienticn'] = train_patient.index
dataframe=dataframe.set_index(['patienticn'])

In [115]:
train_fm = pd.concat([train_fm, dataframe], axis=1)

In [116]:
ohe=pickle.load(open(MODEL_PATH+"Race_encoder", "rb"))
race_encode=ohe.transform(train_patient[["race"]]).toarray()
#ohe.get_feature_names(['race'])

In [117]:
dataframe=pd.DataFrame(race_encode, columns=ohe.get_feature_names(['race'])) 
dataframe['patienticn'] = train_patient.index
dataframe=dataframe.set_index(["patienticn"])
dataframe.index = dataframe.index.map("".join)

In [118]:
train_fm = pd.concat([train_fm, dataframe], axis=1)

In [119]:
le = pickle.load(open(MODEL_PATH+"Ethnicity_encoder", "rb"))
ethnicity_encode=le.transform(train_patient["ethnicity"])

In [120]:
dataframe=pd.DataFrame(ethnicity_encode, columns=["ethnicity"]) 
dataframe['patienticn'] = train_patient.index
dataframe=dataframe.set_index(['patienticn'])

In [121]:
train_fm = pd.concat([train_fm, dataframe], axis=1)

In [122]:
train_fm = pd.concat([train_fm, train_patient[["ageatindexdate"]]], axis=1)

In [123]:
for column in train_fm:
    if(train_fm[column].isnull().values.any()):
        print(column)

asymptomatic30d


In [124]:
train_fm=train_fm.fillna(0)

In [125]:
train_fm.shape

(18887, 182)

In [126]:
#train_fm.to_csv(MODEL_PATH+"TestMatrix.csv")

In [128]:
use_feature = pickle.load(open(MODEL_PATH+"use_feature", "rb"))

#print(use_feature)

In [129]:
len(use_feature)

73

In [130]:
def model(featmat, MODELPATH, PRED):
  scaler = pickle.load(open(MODEL_PATH+"ScalerModel", "rb"))
  X_num = scaler.transform(featmat)

  pred_out = featmat[[]]
  for label in ['gbm', 'ada', 'lgbm_fpr', 'Ensemble']:
    clf = pickle.load(open(MODELPATH+PRED+label+"_Model", "rb"))
    pred_out.loc[:,label] = clf.predict_proba(X_num)[:,1]
  return pred_out

In [131]:
#alive status
fm_alive = train_fm.loc[train_fm['everpositive'] > 0, use_feature]
train_test_res = []
res = model(fm_alive, MODEL_PATH, "death")
train_test_res.append(res)
train_test_out = pd.concat(train_test_res)

/home/luoqi/projects/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/luoqi/projects/py36/lib/python3.6/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/home/luoqi/projects/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [132]:
#train_test_out.head()

In [133]:
negative_id = train_fm.loc[train_fm['everpositive'] == 0, ['everpositive']]
negative_id.rename(columns={'everpositive':'Outcome'}, inplace=True)

In [134]:
import os
os.system("mkdir "+OUTPUT_PATH+"Ensemble")
os.system("mkdir "+OUTPUT_PATH+"gbm")
os.system("mkdir "+OUTPUT_PATH+"ada")

0

In [135]:
#alive status
T=train_test_out.loc[:, ['Ensemble']]
T=T.rename(columns={'Ensemble':'Outcome'})
T=T.append(negative_id)
T.to_csv(OUTPUT_PATH+"Ensemble/alive_predict.tsv", header=False)
T=train_test_out.loc[:, ['lgbm_fpr']]
T=T.rename(columns={'lgbm_fpr':'Outcome'})
T=T.append(negative_id)
T.to_csv(OUTPUT_PATH+"gbm/alive_predict.tsv", header=False)
T=train_test_out.loc[:, ['ada']]
T=T.rename(columns={'ada':'Outcome'})
T=T.append(negative_id)
T.to_csv(OUTPUT_PATH+"ada/alive_predict.tsv", header=False)

In [136]:
#COVID status
T=train_fm[['everpositive']]
T.to_csv(OUTPUT_PATH+"Ensemble/status_predict.tsv", header=False)
T.to_csv(OUTPUT_PATH+"gbm/status_predict.tsv", header=False)
T.to_csv(OUTPUT_PATH+"ada/status_predict.tsv", header=False)

In [137]:
#VENT status
fm_vent = train_fm.loc[train_fm['everpositive'] > 0, use_feature]
train_test_res = []
res = model(fm_vent, MODEL_PATH, "vent_req")
train_test_res.append(res)
train_test_out = pd.concat(train_test_res)

/home/luoqi/projects/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/luoqi/projects/py36/lib/python3.6/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/home/luoqi/projects/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [138]:
#VENT status
T=train_test_out.loc[:, ['Ensemble']]
T=T.rename(columns={'Ensemble':'Outcome'})
T=T.append(negative_id)
T.to_csv(OUTPUT_PATH+"Ensemble/vent_predict.tsv", header=False)
T=train_test_out.loc[:, ['lgbm_fpr']]
T=T.rename(columns={'lgbm_fpr':'Outcome'})
T=T.append(negative_id)
T.to_csv(OUTPUT_PATH+"gbm/vent_predict.tsv", header=False)
T=train_test_out.loc[:, ['ada']]
T=T.rename(columns={'ada':'Outcome'})
T=T.append(negative_id)
T.to_csv(OUTPUT_PATH+"ada/vent_predict.tsv", header=False)

In [139]:
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [140]:
X_num = train_fm.loc[train_fm['everpositive'] > 0, use_feature].values
scaler = pickle.load(open(MODEL_PATH+"ScalerModel", "rb"))
X_num = scaler.transform(X_num)

In [141]:
#days in ICU
clf_gbm_final=pickle.load(open(MODEL_PATH+"days_icu_gbm_Model", "rb"))

Ytest_gbm_predict=train_fm.loc[train_fm['everpositive'] > 0, []]
Y_predict = clf_gbm_final.predict(X_num)
Y_predict[Y_predict<0] = 0
Ytest_gbm_predict.loc[:,'Outcome'] = Y_predict
Ytest_gbm_predict=Ytest_gbm_predict.append(negative_id)
Ytest_gbm_predict.to_csv(OUTPUT_PATH+"gbm/icu_predict.tsv", header=False)

In [142]:
clf_rf_final=pickle.load(open(MODEL_PATH+"days_icu_rf_Model", "rb"))

Ytest_rf_predict=train_fm.loc[train_fm['everpositive'] > 0, []]
Y_predict = clf_rf_final.predict(X_num)
Y_predict[Y_predict<0] = 0
Ytest_rf_predict.loc[:,'Outcome'] = Y_predict
Ytest_rf_predict=Ytest_rf_predict.append(negative_id)
Ytest_rf_predict.to_csv(OUTPUT_PATH+"ada/icu_predict.tsv", header=False)

In [143]:
clf_VR2_final=pickle.load(open(MODEL_PATH+"days_icu_Ensemble_Model", "rb"))

Ytest_VR2_predict=train_fm.loc[train_fm['everpositive'] > 0, []]
Y_predict = clf_VR2_final.predict(X_num)
Y_predict[Y_predict<0] = 0
Ytest_VR2_predict.loc[:,'Outcome'] = Y_predict
Ytest_VR2_predict=Ytest_VR2_predict.append(negative_id)
Ytest_VR2_predict.to_csv(OUTPUT_PATH+"Ensemble/icu_predict.tsv", header=False)

In [144]:
#days hospitalized
clf_gbm_final=pickle.load(open(MODEL_PATH+"days_hospitalized_gbm_Model", "rb"))

Ytest_gbm_predict=train_fm.loc[train_fm['everpositive'] > 0, []]
Y_predict = clf_gbm_final.predict(X_num)
Y_predict[Y_predict<0] = 0
Ytest_gbm_predict.loc[:,'Outcome'] = Y_predict
Ytest_gbm_predict=Ytest_gbm_predict.append(negative_id)
Ytest_gbm_predict.to_csv(OUTPUT_PATH+"gbm/days_hospitalized_predict.tsv", header=False)

In [145]:
clf_rf_final=pickle.load(open(MODEL_PATH+"days_hospitalized_rf_Model", "rb"))

Ytest_rf_predict=train_fm.loc[train_fm['everpositive'] > 0, []]
Y_predict = clf_rf_final.predict(X_num)
Y_predict[Y_predict<0] = 0
Ytest_rf_predict.loc[:,'Outcome'] = Y_predict
Ytest_rf_predict=Ytest_rf_predict.append(negative_id)
Ytest_rf_predict.to_csv(OUTPUT_PATH+"ada/days_hospitalized_predict.tsv", header=False)

In [146]:
clf_VR2_final=pickle.load(open(MODEL_PATH+"days_hospitalized_Ensemble_Model", "rb"))

Ytest_VR2_predict=train_fm.loc[train_fm['everpositive'] > 0, []]
Y_predict = clf_VR2_final.predict(X_num)
Y_predict[Y_predict<0] = 0
Ytest_VR2_predict.loc[:,'Outcome'] = Y_predict
Ytest_VR2_predict=Ytest_VR2_predict.append(negative_id)
Ytest_VR2_predict.to_csv(OUTPUT_PATH+"Ensemble/days_hospitalized_predict.tsv", header=False)